In [1]:
from langchain_chroma import Chroma
from langchain_openai import OpenAIEmbeddings
from langchain_openai import ChatOpenAI
from langchain_core.runnables import RunnablePassthrough
from langchain_core.prompts.few_shot import FewShotPromptTemplate
from langchain_core.prompts import PromptTemplate
from langchain_core.runnables import RunnableMap, RunnablePassthrough

import numpy as np

import re
import json

In [2]:
from dotenv import load_dotenv
load_dotenv()

True

In [3]:
def convert_json(st):
    try:
        st = re.search(r'\{.*?\}', st, re.DOTALL).group(0)
        st = json.loads(st)
        return st
    except:
        return False

In [8]:
vectorstore = Chroma(embedding_function=OpenAIEmbeddings(), persist_directory='C:/project4/chat/testDB')
retriever = vectorstore.as_retriever(search_kwargs={"k":3})

# page_content만 저장
def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)


chatgpt = ChatOpenAI(
    model_name="gpt-4o-mini",
    temperature = 1
)

In [4]:
examples = [
    {
        "question": "드라마의 특성으로 거리가 먼 것은?",
        "answer" :"주로 문자를 통하여 내용이 전달된다.",
    },
    {
        "question": "독도가 대한민국 영토임을 알리는 이유로 알맞은 것은",
        "answer": "독도가 오랫동안 대한민국의 행정 구역으로 관리되어 왔기 때문이다."
    },
    {
        "question": "소설 *소나기*에서 소년이 소녀에게 특별한 감정을 느끼게 된 계기로 거리가 가까운 것은?",
        "answer": "둘이 함께 소나기를 피하면서"
    }
]

example_prompt = PromptTemplate.from_template(
    "'ques' : '{question}', 'ans': '{answer}'" # examples랑 같아야 함
)

In [5]:
example_prompt.format(**examples[0])

"'ques' : '드라마의 특성으로 거리가 먼 것은?', 'ans': '주로 문자를 통하여 내용이 전달된다.'"

In [14]:
prompt = FewShotPromptTemplate(
    examples=examples,
    example_prompt=example_prompt,
    prefix = "당신은 선생님입니다. {context}를 기반으로 정확하게 {theme}에 대한 객관식으로 선택할 수 있는 문제를 하나 만들고 질문에 대한 정답을 JSON 형식으로 만들어주세요.",
    suffix="'ques': '생성된 문제', 'ans': '정답'",
    input_variables=["theme"],
)

In [15]:
rag_chain = (
    {'context': retriever|format_docs, 'theme': RunnablePassthrough()} # chat_prompt가 갖는 dict
    |prompt # 프롬프트
    |chatgpt # 모델
)

In [16]:
ans = rag_chain.invoke('해양쓰레기 종류')

In [17]:
ans.content

'```json\n{\n  "ques": "해양쓰레기의 육상 기인 쓰레기가 아닌 것은?",\n  "ans": "어구와 낚시 용품"\n}\n```'

In [18]:
js = convert_json(ans.content)

In [19]:
js

{'ques': '해양쓰레기의 육상 기인 쓰레기가 아닌 것은?', 'ans': '어구와 낚시 용품'}

In [36]:
val_examples = [
    {
        "question":"해양쓰레기 종류에 대한 설명으로 알맞은 것은?",
        "answer": "해양쓰레기는 육상과 해상 모두에서 발생할 수 있다.",
        "corr_1": "해양쓰레기는 오직 플라스틱으로만 구성된다.",
        "corr_2": "해양쓰레기는 바다 생물의 생태계에 직접적인 영향을 미치지 않는다.",
        "corr_3": "해양쓰레기는 모두 수거되어 재활용된다.",
    },
    {
        "question": "해양쓰레기가 발생하는 주된 원인은 무엇인가?",
        "answer": "하천과 강을 따라 바다로 들어오는 쓰레기", 
        "corr_1": "해양 동물의 생태계를 보호하기 위한 법제정", 
        "corr_2": "멸종 위기 종의 증가",
        "corr_3": "자원봉사자의 봉사활동",
    }
]
val_example_prompt = PromptTemplate.from_template(
    "문제: {question}\n정답: {answer}\n"
    "오답: 'wrong1': '{corr_1}', 'wrong2': '{corr_2}', 'wrong3': '{corr_3}'"
)

In [37]:
val_example_prompt.format(**val_examples[0])

"문제: 해양쓰레기 종류에 대한 설명으로 알맞은 것은?\n정답: 해양쓰레기는 육상과 해상 모두에서 발생할 수 있다.\n오답: 'wrong1': '해양쓰레기는 오직 플라스틱으로만 구성된다.', 'wrong2': '해양쓰레기는 바다 생물의 생태계에 직접적인 영향을 미치지 않는다.', 'wrong3': '해양쓰레기는 모두 수거되어 재활용된다.'"

In [38]:
print(val_example_prompt.format(**val_examples[0]))

문제: 해양쓰레기 종류에 대한 설명으로 알맞은 것은?
정답: 해양쓰레기는 육상과 해상 모두에서 발생할 수 있다.
오답: 'wrong1': '해양쓰레기는 오직 플라스틱으로만 구성된다.', 'wrong2': '해양쓰레기는 바다 생물의 생태계에 직접적인 영향을 미치지 않는다.', 'wrong3': '해양쓰레기는 모두 수거되어 재활용된다.'


In [42]:
valid_prompt = FewShotPromptTemplate(
    examples=val_examples,
    example_prompt=val_example_prompt,
    prefix="당신은 다음의 문제를 보고, 문제에 대한 오답 선지를 만드는 사람입니다. 오해의 소지가 있을 문구는 제외하여 오답을 만들고 JSON 형식으로 만들어주세요.",
    suffix= "문제: {ques}\n정답: {ans}\n오답: ",
    input_variables=["ques", "ans"],
)

In [43]:
valid_prompt.format(ques=js['ques'], ans=js['ans'])

"당신은 다음의 문제를 보고, 문제에 대한 오답 선지를 만드는 사람입니다. 오해의 소지가 있을 문구는 제외하여 오답을 만들고 JSON 형식으로 만들어주세요.\n\n문제: 해양쓰레기 종류에 대한 설명으로 알맞은 것은?\n정답: 해양쓰레기는 육상과 해상 모두에서 발생할 수 있다.\n오답: 'wrong1': '해양쓰레기는 오직 플라스틱으로만 구성된다.', 'wrong2': '해양쓰레기는 바다 생물의 생태계에 직접적인 영향을 미치지 않는다.', 'wrong3': '해양쓰레기는 모두 수거되어 재활용된다.'\n\n문제: 해양쓰레기가 발생하는 주된 원인은 무엇인가?\n정답: 하천과 강을 따라 바다로 들어오는 쓰레기\n오답: 'wrong1': '해양 동물의 생태계를 보호하기 위한 법제정', 'wrong2': '멸종 위기 종의 증가', 'wrong3': '자원봉사자의 봉사활동'\n\n문제: 해양쓰레기의 육상 기인 쓰레기가 아닌 것은?\n정답: 어구와 낚시 용품\n오답: "

In [44]:
print(valid_prompt.format(ques=js['ques'], ans=js['ans']))

당신은 다음의 문제를 보고, 문제에 대한 오답 선지를 만드는 사람입니다. 오해의 소지가 있을 문구는 제외하여 오답을 만들고 JSON 형식으로 만들어주세요.

문제: 해양쓰레기 종류에 대한 설명으로 알맞은 것은?
정답: 해양쓰레기는 육상과 해상 모두에서 발생할 수 있다.
오답: 'wrong1': '해양쓰레기는 오직 플라스틱으로만 구성된다.', 'wrong2': '해양쓰레기는 바다 생물의 생태계에 직접적인 영향을 미치지 않는다.', 'wrong3': '해양쓰레기는 모두 수거되어 재활용된다.'

문제: 해양쓰레기가 발생하는 주된 원인은 무엇인가?
정답: 하천과 강을 따라 바다로 들어오는 쓰레기
오답: 'wrong1': '해양 동물의 생태계를 보호하기 위한 법제정', 'wrong2': '멸종 위기 종의 증가', 'wrong3': '자원봉사자의 봉사활동'

문제: 해양쓰레기의 육상 기인 쓰레기가 아닌 것은?
정답: 어구와 낚시 용품
오답: 


In [45]:
val_chain = (
    RunnableMap({
        'ques': RunnablePassthrough(),
        'ans': RunnablePassthrough(),
    })
    |valid_prompt | chatgpt 
)

In [46]:
js

{'ques': '해양쓰레기의 육상 기인 쓰레기가 아닌 것은?', 'ans': '어구와 낚시 용품'}

In [47]:
output = val_chain.invoke(js)

In [48]:
output.content

'```json\n{\n  "wrong1": "플라스틱 병",\n  "wrong2": "종이컵",\n  "wrong3": "음식물 쓰레기"\n}\n```'

In [49]:
result = convert_json(output.content)

In [50]:
result

{'wrong1': '플라스틱 병', 'wrong2': '종이컵', 'wrong3': '음식물 쓰레기'}

In [51]:
js.update(result)

In [52]:
js

{'ques': '해양쓰레기의 육상 기인 쓰레기가 아닌 것은?',
 'ans': '어구와 낚시 용품',
 'wrong1': '플라스틱 병',
 'wrong2': '종이컵',
 'wrong3': '음식물 쓰레기'}

In [53]:
import random

options = [js['ans'], js['wrong1'], js['wrong2'], js['wrong3']]
random.shuffle(options)

print(options)

['플라스틱 병', '종이컵', '어구와 낚시 용품', '음식물 쓰레기']


In [55]:
options.index(js['ans'])

2

In [56]:
quiz = {"문제": js['ques'], "n1":options[0], "n2":options[1], "n3": options[2], "n4": options[3], "정답": options.index(js['ans'])+1}

In [59]:
quiz

{'문제': '해양쓰레기의 육상 기인 쓰레기가 아닌 것은?',
 'n1': '플라스틱 병',
 'n2': '종이컵',
 'n3': '어구와 낚시 용품',
 'n4': '음식물 쓰레기',
 '정답': 3}

# test

In [61]:
ans = val_chain.invoke({'ques': '해양쓰레기가 발생하는 주요 원인으로 올바르지 않은 것은?', 'ans': '3'})

In [62]:
ans.content

'```json\n{\n  "wrong1": "해양쓰레기를 줄이기 위한 국제 협약의 강화",\n  "wrong2": "해양 생물의 서식지 보호를 위한 노력",\n  "wrong3": "해양 생태계 복원 프로젝트의 진행"\n}\n```'

In [63]:
convert_json(ans.content)

{'wrong1': '해양쓰레기를 줄이기 위한 국제 협약의 강화',
 'wrong2': '해양 생물의 서식지 보호를 위한 노력',
 'wrong3': '해양 생태계 복원 프로젝트의 진행'}